# TF-IDF Retrieval

### Step 1: Import everything and load variables

In [ ]:
#pip install python-terrier

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib64/jvm/java"
os.environ["PYTERRIER_VERSION"] = "snapshot"
os.environ["PYTERRIER_HELPER_VERSION"] = "snapshot"
import pyterrier as pt
pt.init()

In [ ]:
#import pyterrier as pt

import pandas as pd
from tira.third_party_integrations import ensure_pyterrier_is_loaded, get_input_directory_and_output_directory, persist_and_normalize_run
import json
from tqdm import tqdm

ensure_pyterrier_is_loaded()
input_directory, output_directory = get_input_directory_and_output_directory('./iranthology-dataset-tira')


### Step 2: Load the Data

In [ ]:
print('Step 2: Load the data.')

queries = pt.io.read_topics('milestone-1-topics.xml', format='trecxml')

documents = [json.loads(i) for i in open(input_directory + '/documents.jsonl', 'r')]
documents = [{'docno': i['docno'], 'text': i['text'], 'title': i['original_document']['title'], 'abstract': i['original_document']['abstract']} for i in documents]

print('We look at the first document:\n')
print(documents[0])

### Step 3: Create the Index

In [ ]:
print('Step 3: Create the Index.')

!rm -Rf ./index
iter_indexer = pt.IterDictIndexer("./index", meta={'docno' : 100, 'title': 10240, 'abstract': 10240, 'text': 10240}, blocks=True)
index_ref = iter_indexer.index(tqdm(documents))

### Step 4: Create Retrieval Pipeline

We wanted to try the tf-idf retrieval process. TF-IDF (term frequency - inverse document frequency) is a metric, which tells us how much does a term occur in a document in comparison to in how many documents it exists. In this way, if a term occurs very often in a document, but doesn't occur in very many documents, we can see that the term has a particular meaning for this document (--> theme)

In [ ]:
tfidf = pt.BatchRetrieve(index_ref, wmodel="TF_IDF", verbose=True, metadata=['docno', 'text', 'title', 'abstract'])

tfidf_title = pt.text.scorer(body_attr="title", wmodel="TF_IDF")
tfidf_abstract = pt.text.scorer(body_attr="abstract", wmodel="TF_IDF")
tfidf_text = pt.text.scorer(body_attr="text", wmodel="TF_IDF")

# While trying out different weights the scores didn't change at all. so it didn't seem to make much of a diffenrence

combined_tfidf_score = ((2*tfidf_title) + (0.5*tfidf_abstract) + (1*tfidf_text))

#We decreased the top 1000 results to top 100. Only the most relevant documents are important
#a user would check only the first results anyways.
retrieval_pipeline = tfidf %100 >> combined_tfidf_score 


In [ ]:
print('We show the first ten query document pairs after TF-IDF retrieval to show what fields we have added:')
run = tfidf(queries[:1])
run.head(5)

### Step 5: Create the run

In [ ]:
print('Step 5: Create Run.')

run = retrieval_pipeline(queries)

In [ ]:
print('We look at the first 10 results of the run (query has ben expanded):\n')
run.head(10)

### Step 6: Persist Run

In [ ]:
print('Step 6: Persist Run.')

persist_and_normalize_run(run, output_file=output_directory, system_name='multi-field', depth=1000)

print('Done :)')